In [4]:
import numpy as np
import pandas as pd
datos = pd.read_csv("../Entrenamieto_ECI_2020.csv")

gp = datos[["Bureaucratic_Code_0_Approval",
            "Bureaucratic_Code_0_Approved",
            "Pricing, Delivery_Terms_Quote_Appr",
            "Pricing, Delivery_Terms_Approved",
            "Stage"]]
gp.columns = ['Burocratico_No_Necesario', 
              'Burocratico_Aprobado', 
              'Terminos_No_Necesarios', 
              'Terminos_Aprobados', 
              'Exito']
#Vemos que no hay nulos en las filas
gp.isnull().sum()

Burocratico_No_Necesario    0
Burocratico_Aprobado        0
Terminos_No_Necesarios      0
Terminos_Aprobados          0
Exito                       0
dtype: int64

In [5]:
gp.Exito = (gp.Exito == 'Closed Won').astype(int)
gp

,Burocratico_No_Necesario,Burocratico_Aprobado,Terminos_No_Necesarios,Terminos_Aprobados,Exito
0,1,1,1,1,0
1,0,0,0,0,1
2,0,0,0,0,1
3,1,0,1,0,0
4,1,0,1,0,0
...,...,...,...,...,...
16942,1,1,1,1,1
16943,1,1,1,1,1
16944,1,1,1,1,1
16945,1,1,1,1,0


In [6]:
#Vemos que si no es necesario el codigo burocratico entonces aprobado se setea en 0
((gp['Burocratico_No_Necesario'] == 0) & (gp['Burocratico_Aprobado'] == 1)).sum()

0

In [7]:
#Lo mismo sucede con los terminos
((gp['Terminos_No_Necesarios'] == 0) & (gp['Terminos_Aprobados'] == 1)).sum()

0

In [8]:
gp['Burocraticos'] = gp['Burocratico_No_Necesario'] + gp['Burocratico_Aprobado']
gp['Terminos'] = gp['Terminos_No_Necesarios'] + gp['Terminos_Aprobados']
gp = gp.loc[:, ['Burocraticos', 'Terminos', 'Exito']]
gp

,Burocraticos,Terminos,Exito
0,2,2,0
1,0,0,1
2,0,0,1
3,1,1,0
4,1,1,0
...,...,...,...
16942,2,2,1
16943,2,2,1
16944,2,2,1
16945,2,2,0


In [9]:
distribucion_burocraticos = gp['Burocraticos'].value_counts(normalize = True)
distribucion_burocraticos

0    0.524518
2    0.318876
1    0.156606
Name: Burocraticos, dtype: float64

In [10]:
#A partir de ahora voy a ir extrayendo los datos del df y pasandolos a listas para construir el grafico manualmente.
#La notacion es la siguiente: El nombre de la lista representa al nodo y el indice el link que debe tomar siguiendo 
# la siguiente notacion
#0 : No necesario
#1 : No aprobado
#2 : Aprobado


oportunidades = [0, 0, 0]
oportunidades[0] = distribucion_burocraticos.loc[0]
oportunidades[1] = distribucion_burocraticos.loc[1]
oportunidades[2] = distribucion_burocraticos.loc[2]
oportunidades

[0.5245176137369446, 0.15660588894789637, 0.31887649731515905]

In [11]:
distribucion_terminos = gp.groupby('Burocraticos')['Terminos'].value_counts(normalize = True).to_frame()
buro_no_necesario = [0, 0, 0]
buro_no_aprobado = [0, 0, 0]
buro_aprobado = [0, 0, 0]

buro_no_necesario[0] = distribucion_terminos.loc[(0, 0)]
buro_no_necesario[1] = distribucion_terminos.loc[(0, 1)]
buro_no_necesario[2] = distribucion_terminos.loc[(0, 2)]

buro_no_aprobado[0] = distribucion_terminos.loc[(1, 0)]
buro_no_aprobado[1] = distribucion_terminos.loc[(1, 1)]
buro_no_aprobado[2] = distribucion_terminos.loc[(1, 2)]

buro_aprobado[0] = distribucion_terminos.loc[(2, 0)]
buro_aprobado[1] = distribucion_terminos.loc[(2, 1)]
buro_aprobado[2] = distribucion_terminos.loc[(2, 2)]

In [12]:
distribucion_exito = gp.groupby('Terminos').agg({'Exito' : 'mean'})
termino_no_necesario = [0, 0]
termino_no_aprobado = [0, 0]
termino_aprobado = [0, 0]

termino_no_necesario[0] = distribucion_exito.loc[0]
termino_no_necesario[1] = 1 - distribucion_exito.loc[0]

termino_no_aprobado[0] = distribucion_exito.loc[1]
termino_no_aprobado[1] = 1 - distribucion_exito.loc[1]

termino_aprobado[0] = distribucion_exito.loc[2]
termino_aprobado[1] = 1 - distribucion_exito.loc[2]

distribucion_exito

,Exito
Terminos,
0,0.650891
1,0.265843
2,0.647523


In [13]:
# Se siguio el tutorial de https://towardsdatascience.com/sankey-diagram-basics-with-pythons-plotly-7a13d557401a
import plotly.graph_objects as go

source = [
          0, 0, 0,                       #Oportunidades
          1, 1, 1, 2, 2, 2, 3, 3, 3,     # Codigo Burocratico
          4, 4, 5, 5, 6, 6,              # Terminos de entrega
          7, 8                           # Exito o Fracaso
]                         
target = [
          1, 2, 3,
          4, 5, 6, 4, 5, 6, 4, 5, 6,
          7, 8, 7, 8, 7, 8
]

value = [
oportunidades[2], oportunidades[0], oportunidades[1],                       #De Oportunidades a Burocraticos
buro_aprobado[2], buro_aprobado[0], buro_aprobado[1],                       #De Burocraticos aprobados a Terminos 
buro_no_necesario[2], buro_no_necesario[0], buro_no_necesario[1],           #De Burocraticos no necesarios a Terminos 
buro_no_aprobado[2], buro_no_aprobado[0], buro_no_aprobado[1],              #De Burocraticos no aprobados a Terminos
termino_aprobado[0], termino_aprobado[1],                                   #De Terminos aprobados a Resultado
termino_no_necesario[0], termino_no_necesario[1],                           #De Terminos no necesarios a Resultado
termino_no_aprobado[0], termino_no_aprobado[1]                              #De Terminos no aprobados a Resultado

]              

label = [

'Oportunidades',
'Burocraticos aprobados', 
'Burocratico no necesarios', 
'Burocratico no aprobados', 
'Terminos aprobados', 
'Terminos no necesarios', 
'Terminos no aprobados',
'Exito',
'Fracaso'

]


color_node = [

'#645e61',                          #Nodo Oportunidades
'#645e61', '#645e61', '#645e61',    #Nodos Burocraticos
'#645e61', '#645e61', '#645e61',    #Nodos Terminos
'#645e61', '#645e61'                #Nodos resultados

]


color_link = [

'#85c1e9', '#85c1e9', '#85c1e9',    #De oportunidades a Burocraticos
'#A6E3D7', '#A6E3D7', '#A6E3D7',    #De Burocraticos aprobados a Terminos
'#FEF3C7', '#FEF3C7', '#FEF3C7',    #De Burocraticos no necesarios a Terminos
'#EBBAB5', '#EBBAB5', '#EBBAB5',    #De Burocraticos no aprobados a Terminos
'#afe5ef', '#afe5ef',               #De Terminos aprobados a resultado
'#abeca8', '#abeca8',               #De Terminos no necesarios a resultado
'#f0bbd2', '#f0bbd2',               #De Terminos no aprobados a resultado

]



In [14]:
link = dict(source = source, target = target, value = value, color = color_link)
node = dict(label = label, pad=50, thickness=5, color = color_node)
data = go.Sankey(link = link, node=node)
fig = go.Figure(data)

fig.update_layout(
    font=dict(
        family="Arial, monospace",
        size=18,
        color="Black"
    ),
    title={
        'text': "Distribucion de las oportunidades segun aprobacion burocratica, terminos y exito",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    width=1200, 
    height=600
)

fig.show()